In [1]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


# Welcome to VizieR example workflow

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

**Notes:** 

It is a generic notebook, highlighting what can be done once you chose a catalog. This workflow is suggested by [CDS](https://cdsweb.unistra.fr/) (Strasbourg Astronomical Data Center, house of [VizieR](https://vizier.cds.unistra.fr/viz-bin/VizieR)).

The notebook exploits [pyVO](https://pyvo.readthedocs.io/en/latest/), an advanced library  of The [Virtual Observatory](https://ivoa.net/).

[Astroquery](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html) (not used here) is a well-documented, user-friendly alternative.

--------------------------------------------------------

## 1. Setup

This example notebook has the following dependencies: 

**Required**
- pyvo : this library facilitates the access to the Virtual Observatory (VO) resources. VizieR is part of the VO.
This notebook needs version >=1.4.1
**Optional, for visualization**
- ipyaladin : this is the Aladin-lite sky viewer, bundled as a jupyter widget. It allows to plot catalogs and multi-order coverages (MOC)
- matplotlib : an other option to see catalog points and MOCs

## 2. Metadata exploration with the Virtual Observatory registry

This part uses [pyvo](https://pyvo.readthedocs.io/en) to connect to the VO registry.

In [2]:
# the catalogue name in VizieR
CATALOGUE = "J/A+A/587/A65"

We first retrieve the catalogue information.

In [3]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [18]:
# We can print metadata information about the catalogue
#voresource.describe(verbose=True)
voresource.describe(verbose=True)

Visible and near-infrared solar spectra
Short Name: J/A+A/587/A65
IVOA Identifier: ivo://cds.vizier/j/a+a/587/a65
Access modes: tap#aux, web
- tap#aux: https://tapvizier.cds.unistra.fr/TAPVizieR/tap
- webpage: https://vizier.cds.unistra.fr/viz-bin/VizieR-2?-source=J/A+A/587/A65

We present a new solar flux atlas with the aim of understanding wavelength
precision and accuracy in solar benchmark data. The atlas covers the
wavelength range 405-2300nm and was observed at the Institut fur Astrophysik,
Gottingen (IAG), with a Fourier transform spectrograph (FTS). In contrast to
other FTS atlases, the entire visible wavelength range was observed
simultaneously using only one spectrograph setting. We compare the wavelength
solution of the new atlas to the Kitt Peak solar flux atlases and to the HARPS
frequency-comb calibrated solar atlas. Comparison reveals systematics in the
two Kitt Peak FTS atlases resulting from their wavelength scale construction,
and shows consistency between the IAG and

We can also inspect in details the `resource` object and access the attributes not provided by the describe method. See for example, the first author of a resource: 

In [5]:
voresource.creators[0]

'Reiners A.'

## 3. Access the tabular data of this catalog

We can have a look at the tables available in the catalogue.

In [6]:
tables = voresource.get_tables()
print(f"In this catalogue, we have {len(tables)} tables.")
for table_name, table in tables.items():
    print(f"{table_name}: {table.description}")

In this catalogue, we have 3 tables.
J/A+A/587/A65/tablea1: Absolute convective blueshift of FeI lines measured from the IAG atlas
J/A+A/587/A65/spnir: NIR atlas 1000-2300nm
J/A+A/587/A65/spvis: VIS atlas 405-1065nm


In [7]:
# We can also extract the tables names for later use
tables_names = list(tables.keys())
tables_names

['J/A+A/587/A65/tablea1', 'J/A+A/587/A65/spnir', 'J/A+A/587/A65/spvis']

The actual data can then be accessed using any of the ``access_modes`` of the voresource.

In [8]:
voresource.access_modes()

{'tap#aux', 'web'}

The web access is found by following the ``reference_url``

In [9]:
voresource.reference_url

'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/587/A65'

### 3.1 Execute a SQL/ADQL query

The ``tap#aux`` in the ``access_mode`` response indicates that we can also do a SQL/ADQL query for these VizieR tables.

On the first table of the catalogue, we execute an <a href='https://www.ivoa.net/documents/latest/ADQL.html'>ADQL</a> query.

In [10]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records

<DALResultsTable length=10>
recno  lambda    EW   relDepth  ConvBS
         nm    0.1 pm           m / s 
int32 float64  int16  float64  float64
----- -------- ------ -------- -------
    1 405.1819     --      0.6  -548.7
    2  405.305     79     0.69   -44.4
    3  405.344     60     0.62  -625.3
    4 405.5322     50     0.52  -472.2
    5 405.6181     80     0.74  -597.1
    6 405.7491     --     0.43  -401.4
    7  405.849     46     0.72  -127.7
    8 405.9363    101     0.75  -672.1
    9   405.99     74     0.74  -168.6
   10  406.086     70     0.68  -322.5

In [11]:
tables_names

['J/A+A/587/A65/tablea1', 'J/A+A/587/A65/spnir', 'J/A+A/587/A65/spvis']